# simplest example to expose torch bug

In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam, SGD
import tqdm
import copy
torch.__version__

'1.9.0+cu102'

In [2]:
# random data
n, d = 30, 2
X = torch.randn(n, d)
Y = torch.randn(n)
loader = DataLoader(TensorDataset(X, Y), batch_size=n, shuffle=True)

# simple training code
def train(net, loader, epoch=3):
    
    net.train()
    opt = Adam(net.parameters(), weight_decay=1e-4) # seems like wd is needed to reproduce the error
    criterion = nn.MSELoss()
    for _ in tqdm.trange(epoch):
        for x, y in loader:
            opt.zero_grad()
            o = net(x)
            l = criterion(o, y).mean()
            l.backward()
            opt.step()

net_orig = nn.Linear(d, 1)
train(net_orig, loader)
print([p for p in net_orig.parameters()])
print([p.requires_grad for p in net_orig.parameters()])
print([p.grad for p in net_orig.parameters()])

net = copy.deepcopy(net_orig) # for experiment

  0%|                                                                                                                                                          | 0/3 [00:00<?, ?it/s]/home/jiaxuan/.local/share/virtualenvs/CredibleConceptModel-f6EBRjEz/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 39.83it/s]

[Parameter containing:
tensor([[ 0.3954, -0.5826]], requires_grad=True), Parameter containing:
tensor([-0.4809], requires_grad=True)]
[True, True]
[tensor([[ 0.6346, -0.9830]]), tensor([-1.4661])]


In [3]:
class Sequential_no_grad(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1
        self.net2 = net2
    
    def forward(self, x):
        with torch.no_grad():
            self.net1.eval()
            o = self.net1(x)
        return self.net2(o)
    
def train_on_top(net, no_grad=False):
    if no_grad:
        net2 = Sequential_no_grad(net, nn.Linear(1, 1))
    else:
        net2 = nn.Sequential(net, nn.Linear(1, 1))
    train(net2, loader)
    print([p for p in net.parameters()])
    print([p.requires_grad for p in net.parameters()])
    print([p.grad for p in net.parameters()])
    
train_on_top(net)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 539.67it/s]

[Parameter containing:
tensor([[ 0.3924, -0.5796]], requires_grad=True), Parameter containing:
tensor([-0.4779], requires_grad=True)]
[True, True]
[tensor([[ 0.4468, -0.7045]]), tensor([-2.3036])]


In [4]:
for p in net.parameters():
    p.requires_grad = False
    # p.grad=None # with this also work

train_on_top(net, no_grad=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 537.73it/s]

[Parameter containing:
tensor([[ 0.3894, -0.5766]]), Parameter containing:
tensor([-0.4749])]
[False, False]
[tensor([[0., 0.]]), tensor([0.])]


Note that the above cell is supposed to be equal to the cell before, but that is not the case!!! this is a bug on pytorch end!

In [5]:
for p in net.parameters():
    p.grad=None

train_on_top(net, no_grad=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 519.03it/s]

[Parameter containing:
tensor([[ 0.3894, -0.5766]]), Parameter containing:
tensor([-0.4749])]
[False, False]
[None, None]


turn grad to None also works

In [6]:
net_copy = copy.deepcopy(net_orig)
for p in net_copy.parameters():
    p.requires_grad = False

train_on_top(net_copy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 669.59it/s]

[Parameter containing:
tensor([[ 0.3954, -0.5826]]), Parameter containing:
tensor([-0.4809])]
[False, False]
[None, None]


Apparrently copying the network works!

In [7]:
torch.save(net_orig, 'debug.pt')
net_copy = torch.load('debug.pt')
for p in net_copy.parameters():
    p.requires_grad = False

train_on_top(net_copy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 741.26it/s]

[Parameter containing:
tensor([[ 0.3954, -0.5826]]), Parameter containing:
tensor([-0.4809])]
[False, False]
[None, None]


save and load also doesn't get affected!

In [8]:
net_copy = copy.deepcopy(net_orig)
train_on_top(net_copy, no_grad=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 591.08it/s]

[Parameter containing:
tensor([[ 0.3954, -0.5826]], requires_grad=True), Parameter containing:
tensor([-0.4809], requires_grad=True)]
[True, True]
[None, None]


no grad with copy also works!